<center><h2><b>TP1: Basic Text Processing:</br>Regular Expressions and Edit Distance Measurement<b><h2><center/>

<h3><strong>Exercise 2:</strong></h3>
<p>We want to create a simple spell checker for the English language that relies on a lexicon to correct and suggest corresponding corrections. The operating principle of this spell checker is quite straightforward:</p>
<ul>
   <li>We need to traverse a text, comparing it with the words in the lexicon.
   <li>Words that belong to the lexicon are considered correct.
   <li>For unrecognized words, we calculate the "Levenshtein distance" with the words from the lexicon. The word with the smallest distance is considered the correct spelling. To simplify the task, we will compare incorrect words of length l1 with words of length l2, where l2<=l1+2.
</ul>
<p>At the end, you should compare the corrected text with the reference text and provide the values of recall and precision.</p>

### **Imported Libraries**

In [202]:
import numpy as np
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
import nltk
import re
from sklearn.metrics import precision_score, recall_score, f1_score

nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### **Processing Phase**

In [203]:
def read_file(filename, for_lexicons=False):
    """
    Reads and processes a text file.

    Args:
        filename (str): The name of the file to be read.
        for_lexicons (bool, optional): If True, the function returns a list of lines with leading and trailing
            whitespace removed. If False (default), it tokenizes the content using a WhitespaceTokenizer.

    Returns:
        list or str: Depending on the 'for_lexicons' parameter
    """
    if for_lexicons:
        # Read the file and return a list of lines
        with open(filename, 'r') as f:
            return list((row.strip() for row in f))
    else:
        # Read the file and tokenize its content using WhitespaceTokenizer
        with open(filename, 'r') as f:
            return WhitespaceTokenizer().tokenize(f.read())

In [204]:
# Read and process files
english_voc = read_file('./englishvoc.txt', True)

original_tokens = read_file('./text.txt')
original_tokens = [word.lower() for word in original_tokens]

ref_tokens = read_file('./ref.txt')
ref_tokens = [word.lower() for word in ref_tokens]

In [205]:
def distance_levenshtein(word1, word2):
    """
    Calculate the Levenshtein distance between two words.

    Args:
        word1 (str): The first word for distance calculation.
        word2 (str): The second word for distance calculation.

    Returns:
        int: The Levenshtein distance between the two input words.
    """
    # Create a matrix to store Levenshtein distances
    matrix = np.zeros(shape=(len('#' + word2), len('#' + word1)))

    # Initialize the first row and column of the matrix
    matrix[0, :] = np.array(range(len('#' + word1)))
    for row in range(matrix.shape[0]):
        matrix[row, 0] = row

    # Calculate Levenshtein distances
    for row in range(1, matrix.shape[0]):
        for col in range(1, matrix.shape[1]):
            if word1[col - 1] == word2[row - 1]:
                matrix[row, col] = min([matrix[row - 1, col], matrix[row - 1, col - 1], matrix[row, col - 1]])
            else:
                matrix[row, col] = min([matrix[row - 1, col], matrix[row - 1, col - 1], matrix[row, col - 1]]) + 1

    # Return the Levenshtein distance between the two words
    return int(matrix[-1, -1])

In [206]:
# Verify the calculation
print(distance_levenshtein('diet', 'det'))
distance_levenshtein('diet', 'deit')

1


2

In [207]:
def find_words_by_length(word, english_voc):
    """
    Find words in the English vocabulary that match a specific length.

    Args:
        word (str): The input word to compare word lengths with.
        english_voc (list): A list of words in the English vocabulary.

    Returns:
        list: A list of words from the English vocabulary that have similar lengths or within ±2 characters to the input word.
    """
    word_length = len(word)
    result = set()

    for w in english_voc:
        # Check if the length of word 'w' is within ±2 characters of the input word's length
        if word_length - 2 <= len(w) <= word_length + 2:
            result.add(w)

    # Exclude the input word from the result
    result.discard(word)

    # Convert the result set to a list for easier handling
    return list(result)

In [208]:
def contains_only_english_letters(word):
    """
    Check if a word contains only English letters.

    Args:
        word (str): The word to be checked.

    Returns:
        bool: True if the word contains only English letters, False otherwise.
    """
    # Use a regular expression to check if the word contains only English letters (uppercase or lowercase)
    return bool(re.match("^[a-zA-Z]+$", word))

In [209]:
def eliminate_a_character(words):
    """
    Eliminate words consisting of a single character or stacked punctuation.

    Args:
        words (list): A list of words.

    Returns:
        list: A list of words with single-character words and stacked punctuation removed.
    """
    # Define a regular expression pattern to match words with stacked punctuation or a single character
    stacked_punctuation_pattern = r'^[\W_]+$'

    # Use list comprehensions to filter out words with stacked punctuation or single characters
    proper_words = [word for word in words if not re.match(stacked_punctuation_pattern, word)]
    
    # Further filter to remove words with a length of 1 (single characters)
    altered_words = [word for word in proper_words if len(word) > 1]
    
    return altered_words

In [210]:
def suggest_transposition_correction(word, english_voc):
    """
    Suggest a transposition correction for a word.

    Args:
        word (str): The input word to check for transposition.
        ref_word (str): The reference word to compare against.

    Returns:
        bool: True if a transposition correction is suggested, False otherwise.
    """
    # Check for transpositions by comparing characters in adjacent positions
    for i in range(len(word) - 1):
        candidate = word[:i] + word[i+1] + word[i] + word[i+2:]
        if candidate in english_voc:
            return candidate

    return word

In [211]:
def calculate_distances_and_get_closest_word(word, words, english_voc):
    """
    Calculate Levenshtein distances between a word and a list of words and return the closest word.

    Args:
        word (str): The input word to be compared.
        words (list): A list of words to compare against.
        ref_word (str): The reference word for comparison.
        english_voc (list): A list of words in the English vocabulary.

    Returns:
        str: The closest word to the input word based on Levenshtein distance calculations.
    """
    # Check whether the transposition processing helps in achieving this right word or not
    transposed_word = suggest_transposition_correction(word, english_voc)
    if transposed_word != word:
        return transposed_word
    
    distances = []

    # Calculate Levenshtein distances between the input word and each word in the list
    for w in words:
        distances.append(distance_levenshtein(word, w))

    # Create a DataFrame to store words and their distances
    df = pd.DataFrame({'Word': words, 'Distance': distances})

    # Filter words with the minimum distance
    sorted_df = df[df['Distance'] == df['Distance'].min()].sort_values(by=['Word'], ascending=False)
    
    return sorted_df.iloc[0, 0]  # Return the closest word based on Levenshtein distance

In [212]:
def correct_token(word_pattern, english_voc):
    """
    Correct a word pattern by finding the closest word in English vocabulary for each word element.

    Args:
        word_pattern (list): The list of word elements to be corrected.
        reference_token (str): The reference token for comparison.
        english_voc (list): A list of words in the English vocabulary.

    Returns:
        tuple: A tuple containing the corrected word pattern (as a string) and an auxiliary word.
    """
    
    # Iterate through the word elements in the word pattern
    for p in range(len(word_pattern)):
        word = word_pattern[p]
        # Skip word elements with a length of 1
        if not contains_only_english_letters(word) or len(word) <= 2:
            continue

        # Check if the word contains only English letters
        if contains_only_english_letters(word):
            if word in english_voc:
                continue
            # Find the closest word from the list of words with similar lengths
            words = find_words_by_length(word, english_voc)
            closest_word = calculate_distances_and_get_closest_word(word, words, english_voc)
            word_pattern[p] = closest_word

    # Return the corrected word pattern as a string
    return ''.join(word_pattern)

In [213]:
def calculate_metrics(ref_tokens, original_tokens, zero_division=1):
    """
    Calculate precision, recall, and F1-score for system predictions compared to corrected words.

    Args:
        system_predictions (list): List of words predicted by the system.
        corrected_words (list): List of the corrected words.

    Returns:
        float: Precision score.
        float: Recall score.
        float: F1-score.
    """
    # Compute precision, recall, and F1-score
    precision = precision_score(ref_tokens, original_tokens, average='weighted', zero_division=zero_division)
    recall = recall_score(ref_tokens, original_tokens, average='weighted', zero_division=zero_division)
    f1 = f1_score(ref_tokens, original_tokens, average='weighted', zero_division=zero_division)
    
    return precision, recall, f1

In [214]:
def technical_report(original_token, corrected_word, reference_token, check):
    """
    Print a technical report for a correction check.

    Args:
        original_token (str): The original source token.
        corrected_word (str): The system-corrected word.
        reference_token (str): The reference source token.
        check (int): The check number for identification.
    """
    print(f'------------------- CHECK #{check} -------------------')
    print(f'\t+ Original Source:   {original_token}')
    print(f'\t+ Reference Source:  {reference_token}')
    print(f'\t+ System Correction: {corrected_word}\n')

In [215]:
def start_spelling_checker(original_tokens, ref_tokens, english_voc):
    """
    Perform spelling correction and evaluation on a list of tokens.

    Args:
        original_tokens (list): List of original tokens to be corrected.
        ref_tokens (list): List of reference tokens (corrected tokens).
        english_voc (set): Set of English vocabulary for correction.

    Returns:
        None
    """
    for t in range(len(original_tokens)):
        original_token = original_tokens[t]

        # Extract words and non-words in the token
        word_pattern = re.findall(r'(\w+|[^\w\s]+)', original_token)

        # Handle hyphenated words (e.g., 'high-speed' => 'highspeed')
        if len(word_pattern) == 3 and len(word_pattern[0]) <= 2 and word_pattern[1] == '-':
            word_pattern = [word_pattern[0] + word_pattern[-1]]

        # Correct the token based on the English vocabulary
        original_tokens[t] = correct_token(word_pattern, english_voc)

        # Print a technical report for each token
        technical_report(original_token, original_tokens[t], ref_tokens[t], t + 1)

    # Calculate precision and recall
    precision, recall, f1 = calculate_metrics(ref_tokens, original_tokens)

    # Print precision, recall, and F1-score
    print('**************************')
    print(f'P = {precision}\tR = {recall}\tF1 = {f1}')

In [216]:
start_spelling_checker(original_tokens, ref_tokens, english_voc)

------------------- CHECK #1 -------------------
	+ Original Source:   when
	+ Reference Source:  when
	+ System Correction: when

------------------- CHECK #2 -------------------
	+ Original Source:   the
	+ Reference Source:  the
	+ System Correction: the

------------------- CHECK #3 -------------------
	+ Original Source:   police
	+ Reference Source:  police
	+ System Correction: police



------------------- CHECK #4 -------------------
	+ Original Source:   forciby
	+ Reference Source:  forcibly
	+ System Correction: forcibly

------------------- CHECK #5 -------------------
	+ Original Source:   entred
	+ Reference Source:  entered
	+ System Correction: entree

------------------- CHECK #6 -------------------
	+ Original Source:   43-year-old
	+ Reference Source:  43-year-old
	+ System Correction: 43-year-old

------------------- CHECK #7 -------------------
	+ Original Source:   paulina
	+ Reference Source:  paulina
	+ System Correction: pauline

------------------- CHECK #8 -------------------
	+ Original Source:   fiore’s
	+ Reference Source:  fiore’s
	+ System Correction: fore’s

------------------- CHECK #9 -------------------
	+ Original Source:   apartment,
	+ Reference Source:  apartment,
	+ System Correction: apartment,

------------------- CHECK #10 -------------------
	+ Original Source:   they
	+ Reference Source:  they
	+ System Correction: they

--------

In [217]:
# Called the corpus.words from NLTK as the englishvoc.txt don't contain all the lexicons for the spelling checker program
# nltk.corpus.words.words() is a list of more than 240.000 English lexicons
start_spelling_checker(original_tokens, ref_tokens, nltk.corpus.words.words())

------------------- CHECK #1 -------------------
	+ Original Source:   when
	+ Reference Source:  when
	+ System Correction: when

------------------- CHECK #2 -------------------
	+ Original Source:   the
	+ Reference Source:  the
	+ System Correction: the

------------------- CHECK #3 -------------------
	+ Original Source:   police
	+ Reference Source:  police
	+ System Correction: police

------------------- CHECK #4 -------------------
	+ Original Source:   forcibly
	+ Reference Source:  forcibly
	+ System Correction: forcibly

------------------- CHECK #5 -------------------
	+ Original Source:   entree
	+ Reference Source:  entered
	+ System Correction: entree

------------------- CHECK #6 -------------------
	+ Original Source:   43-year-old
	+ Reference Source:  43-year-old
	+ System Correction: 43-year-old

------------------- CHECK #7 -------------------
	+ Original Source:   pauline
	+ Reference Source:  paulina
	+ System Correction: paulin

------------------- CHECK #8 ---